# Analyzing the frequency components of a signal with a Fast Fourier Transform

Fast Fourier Transforms (FFT) are a fast way to find dominant frequencies in any signal.

Lets start with the standard imports.

In [ ]:
import datetime
import numpy as np
import scipy as sp
import scipy.fftpack
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Now lets look at some weather data to see if we can find a dominant seasonal pattern.

In [ ]:
df0 = pd.read_csv('data/weather.csv',
                  na_values=(-9999),
                  parse_dates=['DATE'])

In [ ]:
df = df0[df0['DATE'] >= '19940101']

In [ ]:
df.head()

In [ ]:
df_avg = df.dropna().groupby('DATE').mean()

In [ ]:
df_avg.head()

In [ ]:
date = pd.to_datetime(df_avg.index)

In [ ]:
temp = (df_avg['TMAX'] + df_avg['TMIN']) / 20.
N = len(temp)

If we plot the temperature results for the last several years, we see a pattern.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 3))
temp.plot(ax=ax, lw=.5)
ax.set_ylim(-10, 40)
ax.set_xlabel('Date')
ax.set_ylabel('Mean temperature')

Now lets apply the FFT to calculate the frequency of that pattern.  Does it match our expected result of peaks in temperature roughly each year?

In [ ]:
temp_fft = sp.fftpack.fft(temp)

In [ ]:
temp_psd = np.abs(temp_fft) ** 2

In [ ]:
fftfreq = sp.fftpack.fftfreq(len(temp_psd), 1. / 365)

In [ ]:
i = fftfreq > 0

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax.plot(fftfreq[i], 10 * np.log10(temp_psd[i]))
ax.set_xlim(0, 5)
ax.set_xlabel('Frequency (1/year)')
ax.set_ylabel('PSD (dB)')

Sure enough there is a dominant frequency at 1.  Therefore we can conclude that there is a perodicity of the data when we look at the annual temperature, and the period is almost exactly 1 year.

Of course we are not suprised.  We know this to be true.

Can you think of other things that might reveal an underlying structure with a perodicity?

Using these results, we can apply the period of 1 year to plot a "smooth" line of mean temperature, overlayed on the actual temperature.  From this we could do futher analysis, but we will stop here.

In [ ]:
temp_fft_bis = temp_fft.copy()
temp_fft_bis[np.abs(fftfreq) > 1.1] = 0

In [ ]:
temp_slow = np.real(sp.fftpack.ifft(temp_fft_bis))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 3))
temp.plot(ax=ax, lw=.5)
ax.plot_date(date, temp_slow, '-')
ax.set_xlim(datetime.date(1994, 1, 1),
            datetime.date(2000, 1, 1))
ax.set_ylim(-10, 40)
ax.set_xlabel('Date')
ax.set_ylabel('Mean temperature')